WINDOW FUNCTION 

In [0]:
from pyspark.sql import *
from pyspark import SparkConf

conf = SparkConf()
conf.set("spark.app.name","targaryen")
conf.set("spark.master","local[*]")
    
spark = SparkSession.builder.config(conf=conf).getOrCreate()  
    
# schema = "user_id long , device_id long , mac_address string , registration_timestamp long"
df = (spark.read
        .format("csv") 
        .option("header","true") 
        .option("inferschema","true") 
        .load("/FileStore/tables/5_gym_logins_*")
)


df1 = df.select(df.mac_address.cast('string'),df.gym.cast('int'),df.login.cast('timestamp'),df.logout.cast('timestamp'))


from pyspark.sql.functions import window,sum,count
df2 = df1.groupBy(window(df1.login,"30 minutes")).agg(count("*"),sum(df1.gym))
display(df2)



login and logout merging using window method and pivot method

with pivot method

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("UserLoginLogout").getOrCreate()

data = [
    (1, 'login', '2024-01-25T10:00:00'),
    (1, 'logout', '2024-01-25T12:00:00'),
    (2, 'login', '2024-01-25T11:00:00'),
    (2, 'logout', '2024-01-25T14:00:00'),
]

schema = ["user_id", "type", "time"]

df = spark.createDataFrame(data, schema=schema)

pivoted_df = df.groupBy("user_id").pivot("type").agg({"time": "max"})

result_df = pivoted_df.select(
    col("user_id"),
    col("login").alias("logintime"),
    col("logout").alias("logouttime")
)

result_df.show(truncate=False)

+-------+-------------------+-------------------+
|user_id|logintime          |logouttime         |
+-------+-------------------+-------------------+
|1      |2024-01-25T10:00:00|2024-01-25T12:00:00|
|2      |2024-01-25T11:00:00|2024-01-25T14:00:00|
+-------+-------------------+-------------------+



In [0]:
from pyspark.sql.functions import max,min
display(df)
temp = df.groupBy("user_id").pivot('type').agg(min('time'))
display(temp)

user_id,type,time
1,login,2024-01-25T10:00:00
1,logout,2024-01-25T12:00:00
2,login,2024-01-25T11:00:00
2,logout,2024-01-25T14:00:00


user_id,login,logout
1,2024-01-25T10:00:00,2024-01-25T12:00:00
2,2024-01-25T11:00:00,2024-01-25T14:00:00


In [0]:
from pyspark.sql.functions import when
df1 = (df.select("*").
    # withColumn('logouttime',expr("case when type == 'logout' then time else 0 end")).
    withColumn('logouttime',when(df.type == 'logout',df.time).otherwise(None)).
    withColumn('logintime',when(df.type == 'login',df.time).otherwise(None))
)

with window method

In [0]:
from pyspark.sql.functions import expr,when,col,max,count,first,last
from pyspark.sql.window import Window as win
print(df1.printSchema())

wn = win.partitionBy('user_id')

df2 = (df1.withColumn('lit',first('logintime').over(wn.orderBy(col('time').asc())))
    .withColumn('lot',first('logouttime').over(wn.orderBy(col('time').desc())))
    .select('user_id','lit','lot')
    .dropDuplicates()
    )


# df2=df1.groupBy("user_id").agg(sum(col('logouttime'),col('logintime')))
display(df2)

root
 |-- user_id: long (nullable = true)
 |-- type: string (nullable = true)
 |-- time: string (nullable = true)
 |-- logouttime: string (nullable = true)
 |-- logintime: string (nullable = true)

None


user_id,lit,lot
1,2024-01-25T10:00:00,2024-01-25T12:00:00
2,2024-01-25T11:00:00,2024-01-25T14:00:00
